In [ ]:
!pip install -q transformers torch scikit-learn pandas numpy

In [ ]:
import pandas as pd
import numpy as np
import torch

from transformers import AutoTokenizer, AutoModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import files
uploaded = files.upload()

df = pd.read_csv(list(uploaded.keys())[0])
df.head()

Saving dataset-merged.csv.zip to dataset-merged.csv.zip


,Unnamed: 0,text,label,wcount
0,0,‘मोदी के शासन के दौरान गंगा’ गंगा नदी नरेन्द्...,1,19
1,1,यह खबर आने से पहले छवि क्रेडिट जस्टिन सुलिवान/...,1,374
2,2,गुलाब गेंद वाल डे-नाइट टेस्ट मैच कप्ता विराट क...,0,20
3,3,उत्तर कोरिया रॉकेट प्रक्षेपण योजनाएं 71 0 15 0...,1,345
4,4,राष्ट्रपति डोनाल्ड ट्रम्प और प्रथम महिला मेलान...,0,180


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = "Sailesh97/Hinvec"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

# Install loguru as it's a required dependency for the model
!pip install loguru

model = AutoModel.from_pretrained(
    model_name,
    trust_remote_code=True
)

model.eval()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.4 MB/s eta 0:00:00


model.safetensors:   0%|          | 0.00/3.76G [00:00<?, ?B/s]

BidirectionalMistralModel(
  (embed_tokens): Embedding(32768, 2048, padding_idx=0)
  (layers): ModuleList(
    (0-15): 16 x MistralDecoderLayer(
      (self_attn): MistralAttention(
        (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
        (k_proj): Linear(in_features=2048, out_features=512, bias=False)
        (v_proj): Linear(in_features=2048, out_features=512, bias=False)
        (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
      )
      (mlp): MistralMLP(
        (gate_proj): Linear(in_features=2048, out_features=7168, bias=False)
        (up_proj): Linear(in_features=2048, out_features=7168, bias=False)
        (down_proj): Linear(in_features=7168, out_features=2048, bias=False)
        (act_fn): SiLUActivation()
      )
      (input_layernorm): MistralRMSNorm((2048,), eps=1e-06)
      (post_attention_layernorm): MistralRMSNorm((2048,), eps=1e-06)
    )
  )
  (norm): MistralRMSNorm((2048,), eps=1e-06)
  (rotary_emb): MistralRotaryEmbedd

In [ ]:
def get_hinvec_embedding(text):
    tokens = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=32
    )

    # ⛔ DO NOT call model(**tokens)
    # ✅ Directly use embedding layer
    with torch.no_grad():
        token_embeddings = model.get_input_embeddings()(tokens["input_ids"])

    # Mean pooling
    embedding = token_embeddings.mean(dim=1)

    return embedding.squeeze().cpu().numpy()

In [ ]:
X = []
y = df["label"].values

for text in df["text"].astype(str):
    emb = get_hinvec_embedding(text)
    X.append(emb)

X = np.array(X)
print(X.shape)

(17124, 2048)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

In [ ]:
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.7646715328467153


In [ ]:
# Select one FAKE news from dataset
fake_news_row = df[df["label"] == 1].iloc[0]

news_text = fake_news_row["text"]

# Generate embedding
news_embedding = get_hinvec_embedding(news_text)

# Predict
prediction = clf.predict([news_embedding])[0]

print("News Text:")
print(news_text)

print("\nModel Prediction:")
if prediction == 1:
    print("🛑 Fake News")
else:
    print("✅ Real News")

News Text:
‘मोदी के शासन के दौरान गंगा’  गंगा नदी नरेन्द्र मोदी के शाशन में कांग्रेस के मुकाबले काफ़ी स्वच्छ है  

Model Prediction:
✅ Real News


In [ ]:
test_news = "यह खबर पूरी तरह से झूठी और भ्रामक है और सरकार इसे छुपा रही है।"

emb = get_hinvec_embedding(test_news)
pred = clf.predict([emb])[0]

print("Prediction:", "🛑 Fake" if pred == 1 else "✅ Real")

Prediction: 🛑 Fake


In [ ]:
test_news = "भारत ने एशिया कप क्रिकेट टूर्नामेंट में जीत हासिल की।"

emb = get_hinvec_embedding(test_news)
pred = clf.predict([emb])[0]

print("Prediction:", "🛑 Fake" if pred == 1 else "✅ Real")

Prediction: ✅ Real
